In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train= pd.read_csv("/kaggle/input/titanic/train.csv")
test= pd.read_csv("/kaggle/input/titanic/test.csv")
x=test.PassengerId
y=train.Survived

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import neighbors
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

In [4]:
encoder = OneHotEncoder(sparse=False)
train=train.drop(columns=['Cabin','PassengerId','Survived', 'Name'], axis=1)
test=test.drop(columns=['Cabin','PassengerId', 'Name'], axis=1)
train['Age'].fillna((train['Age'].mean()), inplace=True) 
test['Age'].fillna((test['Age'].mean()), inplace=True) 
test['Fare'].fillna((test['Fare'].mean()), inplace=True) 
train['Embarked'].fillna('S', inplace=True)
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Ticket    891 non-null    object 
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 55.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Ticket    418 non-null    object 
 6   Fare      418 non-null

In [5]:
s = (train.dtypes == 'object')
object_cols = list(s[s].index)

from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(test[object_cols]))

OH_cols_train.index = train.index
OH_cols_test.index = test.index

num_X_train = train.drop(object_cols, axis=1)
num_X_test = test.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test= pd.concat([num_X_test, OH_cols_test], axis=1)

In [6]:
sklearn.neighbors.KNeighborsClassifier
sklearn.linear_model.LinearRegression
sklearn.ensemble.RandomForestClassifier
sklearn.ensemble.GradientBoostingClassifier
xgb_model = XGBClassifier()

In [7]:
training, valid, ytraining, yvalid = train_test_split(OH_X_train,y,test_size=0.5)

In [8]:
model1=RandomForestClassifier()
model2=XGBClassifier()
model3=neighbors.KNeighborsClassifier()
model4=GradientBoostingClassifier()

In [9]:
pred1=model1.fit(training,ytraining)
pred2=model2.fit(training,ytraining)
pred3=model3.fit(training,ytraining)
pred4=model4.fit(training,ytraining)

In [10]:
pred1=model1.predict(valid)
pred2=model2.predict(valid)
pred3=model3.predict(valid)
pred4=model4.predict(valid)

In [11]:
test_pred1=model1.predict(OH_X_test)
test_pred2=model2.predict(OH_X_test)
test_pred3=model3.predict(OH_X_test)
test_pred4=model4.predict(OH_X_test)

In [12]:
stacked=np.column_stack((pred1,pred2,pred3,pred4))
stacked_test_predictions=np.column_stack((test_pred1,test_pred2,test_pred3,test_pred4))

In [13]:
meta_model=XGBClassifier()
meta_model.fit(stacked,yvalid)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
final=meta_model.predict(stacked_test_predictions)
output = pd.DataFrame({'PassengerId': x, 'Survived': final})
output.to_csv('submission.csv', index=False)

In [15]:
submission = pd.read_csv('submission.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
